# Segmentation and Tracking using IceFloeTracker.jl

In [ ]:
# Setup environment
using Pkg
Pkg.add("IceFloeTracker")
Pkg.add("Images")

In [ ]:
# Load packages
using IceFloeTracker: LopezAcosta2019Tiling, Watkins2025GitHub, FloeTracker, ChainedFilterFunction, MinimumWeightMatchingFunction
using Images: erode, segment_mean, labels_map, SegmentedImage, RGB, mosaicview
using Dates: DateTime

In [ ]:
data_loader = Watkins2025GitHub(; ref="25ba4d46814a5423b65ad675aaec05633d17a37e")
cases = data_loader(c -> c.case_number == 6)
mosaicview([c.modis_truecolor for c in cases], nrow=1)

In [ ]:
segmenter = LopezAcosta2019Tiling.Segment()
tc_images = [c.modis_truecolor for c in cases] # TODO: Update validation dataset so that `cases.modis_truecolor` does this
fc_images = [c.modis_falsecolor for c in cases] # TODO: Update validation dataset so that `cases.modis_falsecolor` does this
lm_images = [c.modis_landmask for c in cases] # TODO: Update validation dataset so that `cases.modis_landmask` does this
segmentation_results = segmenter.(tc_images, fc_images, lm_images)

In [ ]:
cases.metadata

In [ ]:
segmentation_results

In [ ]:
tracker = FloeTracker(
    filter_function=ChainedFilterFunction(), 
    matching_function=MinimumWeightMatchingFunction()
)
tracking_results = tracker(segmentation_results, cases.metadata.pass_time)

TODO: would one of these be a better interface?
- `tracking_results = tracker(segmentation_results, cases.metadata.passtime)`
  TODO: rename start_date to passtime in validation data set